In [1]:
import os
import sys
import cv2

import random

import numpy as np
from tqdm import tqdm
import pickle

import tensorflow as tf
from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D 
from keras.layers import Activation, Dropout, Flatten, Dense
from torch.utils.tensorboard import SummaryWriter
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

%matplotlib inline

/scratch/huangtin_cs537/miniconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/scratch/huangtin_cs537/miniconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/scratch/huangtin_cs537/miniconda3/envs/myenv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/scratch/huan

In [2]:
os.listdir('../data/train')

['cat.2718.jpg',
 'dog.1753.jpg',
 'human616.jpg',
 'cat.1211.jpg',
 'dog.775.jpg',
 'human1,666.jpg',
 'human1,100.jpg',
 'cat.1577.jpg',
 'human170.jpg',
 'cat.952.jpg',
 'dog.1035.jpg',
 'cat.946.jpg',
 'dog.1021.jpg',
 'human1,114.jpg',
 'cat.1563.jpg',
 'human164.jpg',
 'dog.2528.jpg',
 'human2,969.jpg',
 'human602.jpg',
 'dog.761.jpg',
 'cat.1205.jpg',
 'human1,672.jpg',
 'dog.1747.jpg',
 'human2,941.jpg',
 'dog.991.jpg',
 'dog.2266.jpg',
 'human1,882.jpg',
 'human2,799.jpg',
 'dog.749.jpg',
 'cat.2724.jpg',
 'dog.1009.jpg',
 'cat.2042.jpg',
 'dog.2500.jpg',
 'human1,128.jpg',
 'human158.jpg',
 'dog.2514.jpg',
 'cat.2056.jpg',
 'cat.2730.jpg',
 'dog.985.jpg',
 'human2,955.jpg',
 'dog.2272.jpg',
 'human1,896.jpg',
 'cat.1239.jpg',
 'human2,766.jpg',
 'cat.6.jpg',
 'dog.2299.jpg',
 'dog.1790.jpg',
 'cat.2903.jpg',
 'dog.1948.jpg',
 'cat.749.jpg',
 'cat.991.jpg',
 'human2,000.jpg',
 'human2,014.jpg',
 'cat.985.jpg',
 'dog.1784.jpg',
 'cat.2917.jpg',
 'human2,772.jpg',
 'human819.jpg

In [3]:
train_images = os.listdir('../data/train')
test_images = os.listdir('../data/test')

print(len(train_images))
print(len(test_images))

9000
3000


In [6]:
train_images[:10]

['cat.2718.jpg',
 'dog.1753.jpg',
 'human616.jpg',
 'cat.1211.jpg',
 'dog.775.jpg',
 'human1,666.jpg',
 'human1,100.jpg',
 'cat.1577.jpg',
 'human170.jpg',
 'cat.952.jpg']

In [7]:
test_images[:10]

['cat.3406.jpg',
 'human616.jpg',
 'dog.3144.jpg',
 'dog.3622.jpg',
 'human170.jpg',
 'cat.3360.jpg',
 'cat.3374.jpg',
 'dog.3636.jpg',
 'human164.jpg',
 'human602.jpg']

In [4]:
test_images_data = []

for image in tqdm(test_images):
    image_data = cv2.imread('../data/test/' + image)
    
    #Convert to GrayScale
    gray = cv2.cvtColor(image_data, cv2.COLOR_BGR2GRAY)
    
    #convert color from BGR to RGB
    #image_data = cv2.cvtColor(image_data, cv2.COLOR_BGR2RGB)
    
    image_data = cv2.resize(gray, (64, 64))
    
    #turn to only borders
    image_data = cv2.Canny(image_data, 150, 150)
    
    test_images_data.append(image_data)

100%|██████████| 3000/3000 [00:07<00:00, 405.12it/s]


In [5]:
test_images_data = np.array(test_images_data)
test_images_data.shape

(3000, 64, 64)

In [6]:
train_images_data = []
train_images_labels = []

random.shuffle(train_images)

for image in tqdm(train_images):
    image_data = cv2.imread('../data/train/' + image)
    
    #Convert to GrayScale
    gray = cv2.cvtColor(image_data, cv2.COLOR_BGR2GRAY)
    
    #convert color from BGR to RGB
    #image_data = cv2.cvtColor(image_data, cv2.COLOR_BGR2RGB)
    
    image_data = cv2.resize(gray, (64, 64))
    
    #turn to only borders
    image_data = cv2.Canny(image_data, 150, 150)
    
    train_images_data.append(image_data)
    train_images_data.append(image_data[:, ::-1]) #flipped image
    
    
    if image.startswith('cat'):
#         train_images_labels.append(0)
#         train_images_labels.append(0)
        train_images_labels.append([0, 1, 0])
        train_images_labels.append([0, 1, 0])
    elif image.startswith('dog'):
#         train_images_labels.append(1)
#         train_images_labels.append(1)
        train_images_labels.append([1, 0, 0])
        train_images_labels.append([1, 0, 0])
    else:
#         train_images_labels.append(2)
#         train_images_labels.append(2)  
        train_images_labels.append([0, 0, 1])
        train_images_labels.append([0, 0, 1])

100%|██████████| 9000/9000 [00:21<00:00, 420.80it/s]


In [7]:
train_images_data = np.array(train_images_data)
train_images_labels = np.array(train_images_labels)

train_images_data = train_images_data.reshape([-1, 64, 64, 1])

In [12]:
import copy
train_images_data_b = copy.deepcopy(train_images_data) 
train_images_labels_b = copy.deepcopy(train_images_labels)
test_images_data = copy.deepcopy(test_images_data)

In [30]:
train_images_data = train_images_data[:4500]
train_images_labels = train_images_labels_b[:4500]
test_images_data = test_images_data[:1500]

In [44]:
pickle.dump(train_images_data, open('edges_train_images_data.pck', 'wb'))

In [5]:
pickle.dump(test_images_data, open('colored_edges_test_images_data.pck', 'wb'))

NameError: name 'test_images_data' is not defined

In [46]:
pickle.dump(train_images_labels, open('edges_train_images_labels.pck', 'wb'))

In [35]:
train_images_data = pickle.load(open('edges_train_images_data.pck', 'rb'))

In [12]:
train_images_data.shape

(18000, 64, 64, 1)

In [21]:
test_images_data = pickle.load(open('edges_test_images_data.pck', 'rb'))

In [31]:
test_images_data.shape

(1500, 64, 64)

In [23]:
train_images_labels = pickle.load(open('edges_train_images_labels.pck', 'rb'))

In [33]:
train_images_labels.shape

(4500, 3)

In [22]:
model_name = 'baseline_adam_dt_9000' #

In [23]:
model = Sequential() 
model.add(Conv2D(64, (5, 5), input_shape=(64, 64, 1))) 
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.1)) 
  
model.add(Conv2D(32, (3, 3))) 
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(Dropout(0.1)) 
  
model.add(Conv2D(32, (3, 3))) 
model.add(Activation('relu')) 
model.add(MaxPooling2D(pool_size=(2, 2))) 
model.add(Dropout(0.1)) 
  
model.add(Flatten()) 
model.add(Dense(50)) #Increase to see how accuracy performs 
#consider also with 0.1 dropout it went from training with 25 to training with 45

model.add(Activation('relu')) 
model.add(Dropout(0.1)) #compare 0.5 with to 0.1 then 0.2 and 0.3

model.add(Dense(3, activation='sigmoid'))
# Dense(3, activation='softmax') 

model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy']) 

# writer = SummaryWriter(log_dir='./logs2/edge_plt.pth')
tensorboard = TensorBoard(log_dir="logs/{}".format(model_name))
# early_stop = EarlyStopping(monitor='val_loss', patience=8)
checkpoint = ModelCheckpoint(filepath= model_name + '_checkpoint.h5', monitor='val_accuracy', 
                             save_best_only=True)

# model.fit(train_images_data, train_images_labels, epochs=100, validation_split=0.2, 
#           callbacks=[tensorboard, early_stop, checkpoint])
model.fit(train_images_data, train_images_labels, epochs=100, validation_split=0.2, 
          callbacks=[tensorboard, checkpoint])

Train on 14400 samples, validate on 3600 samples
Epoch 1/100
14400/14400 [==============================] - 23s 2ms/step - loss: 0.7734 - acc: 0.7051 - val_loss: 0.4194 - val_acc: 0.7762
Epoch 2/100
   96/14400 [..............................] - ETA: 20s - loss: 0.4286 - acc: 0.7708

/scratch/huangtin_cs537/miniconda3/envs/myenv/lib/python3.6/site-packages/keras/callbacks.py:434: RuntimeWarning: Can save best model only with val_accuracy available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


14400/14400 [==============================] - 22s 2ms/step - loss: 0.4124 - acc: 0.7847 - val_loss: 0.3800 - val_acc: 0.8074
Epoch 3/100
14400/14400 [==============================] - 22s 2ms/step - loss: 0.3693 - acc: 0.8150 - val_loss: 0.3498 - val_acc: 0.8230
Epoch 4/100
14400/14400 [==============================] - 22s 2ms/step - loss: 0.3393 - acc: 0.8322 - val_loss: 0.3210 - val_acc: 0.8419
Epoch 5/100
14400/14400 [==============================] - 22s 2ms/step - loss: 0.3212 - acc: 0.8448 - val_loss: 0.3099 - val_acc: 0.8468
Epoch 6/100
14400/14400 [==============================] - 22s 2ms/step - loss: 0.3059 - acc: 0.8544 - val_loss: 0.3009 - val_acc: 0.8502
Epoch 7/100
14400/14400 [==============================] - 22s 2ms/step - loss: 0.2902 - acc: 0.8631 - val_loss: 0.3051 - val_acc: 0.8466
Epoch 8/100
14400/14400 [==============================] - 22s 2ms/step - loss: 0.2781 - acc: 0.8698 - val_loss: 0.2896 - val_acc: 0.8581
Epoch 9/100
14400/14400 [=====================

In [24]:
model.save('models/' + model_name + '.h5')

In [43]:
train_images_data.shape

(4500, 64, 64, 1)

In [46]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_67 (Conv2D)           (None, 60, 60, 64)        1664      
_________________________________________________________________
activation_90 (Activation)   (None, 60, 60, 64)        0         
_________________________________________________________________
max_pooling2d_67 (MaxPooling (None, 30, 30, 64)        0         
_________________________________________________________________
dropout_48 (Dropout)         (None, 30, 30, 64)        0         
_________________________________________________________________
conv2d_68 (Conv2D)           (None, 28, 28, 32)        18464     
_________________________________________________________________
activation_91 (Activation)   (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_68 (MaxPooling (None, 14, 14, 32)        0         
__________

In [55]:
print(tf.__version__)

1.14.0


In [63]:
tf.executing_eagerly()
m = tf.keras.metrics.TopKCategoricalAccuracy(k=1)
_ = m.update_state([[0, 0, 1], [0, 1, 0]],
                   [[0.1, 0.9, 0.8], [0.05, 0.95, 0]])
m.result().numpy()

AttributeError: 'Tensor' object has no attribute 'numpy'

In [64]:
tf.executing_eagerly()

False